In [0]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,DoubleType,DateType

In [0]:
driver_schema = StructType([
    StructField('driverId', IntegerType(),True),
    StructField('driverRef', StringType(),True),
    StructField('number', IntegerType(),True),
    StructField('code', StringType(),True),
    StructField('name', StructType([StructField('forename', StringType(), True), StructField('surname', StringType(), True)
    ]), True),
    StructField('dob', DateType(),True),
    StructField('nationality', StringType(), True),
    StructField('url', StringType(),True)
])

In [0]:
df = spark.read.json('abfss://udayadbcontainerv1@udayadbsav1.dfs.core.windows.net/RAW-V1/drivers.json', schema=driver_schema)

In [0]:
df.display()

In [0]:
# Fore Nam + Sur Name = Fullname
# Drop URl
from pyspark.sql.functions import current_timestamp, lit, concat,col
df = df.withColumnRenamed('driverid', 'driver_id')\
    .withColumnRenamed('driverRef','driver_Ref') \
    .withColumn('name', concat(col('name.forename'),lit(" "),col('name.surname')))\
    .drop(col('url'))

In [0]:
df.display()

In [0]:
df = df.withColumn("Ingestion_Date", current_timestamp())

In [0]:
df.write.mode('overwrite').parquet("abfss://udayadbcontainerv1@udayadbsav1.dfs.core.windows.net/PROCESSED-V1/drivers/drivers.parquet")